In [ ]:
import cohere
import numpy as np
from cohere.classify import Example

In [ ]:
co = cohere.Client('<YOUR_API_KEY>')

In [ ]:
examples = [Example("The order came 5 days early", "positive"), 
    Example("The item exceeded my expectations", "positive"), 
    Example("I ordered more for my friends", "positive"), 
    Example("I would buy this again", "positive"), 
    Example("I would recommend this to others", "positive"), 
    Example("The package was damaged", "negative"), 
    Example("The order is 5 days late", "negative"), 
    Example("The order was incorrect", "negative"), 
    Example("I want to return my item", "negative"), 
    Example("The item\'s material feels low quality", "negative")]

In [ ]:
positive_test_input = "This item was great"
negative_test_input = "This item broke after 3 weeks"

In [ ]:
def cohere_model_wrapper_for_shap(inputs):
  response = co.classify(
    model='small',
    inputs=inputs.tolist(),
    examples=examples)

  confidences = np.zeros((len(inputs), 2))
  
  for i in range(len(inputs)):    
    c = response.classifications[i]
    confidences[i][0] = c.labels['negative'].confidence;
    confidences[i][1] = c.labels['positive'].confidence;  

  return confidences

In [ ]:
import shap

masker = shap.maskers.Text(r"\W")
shapExplainer = shap.Explainer(cohere_model_wrapper_for_shap, masker)

In [ ]:
shap_values = shapExplainer([positive_test_input, negative_test_input])

In [ ]:
shap.plots.text(shap_values)